# Fixing FIPS codes in Storm Dataset

This notebook will serve to see how to fix the fpid problem in "Storm" dataframe, 
which was found to have fips that are not registered. 

## Imports

In [1]:
import geopandas as gpd
import pandas as pd
import warnings


from difflib import SequenceMatcher


import utils_dhm as ut

## Constants

In [6]:
FIPS_PATH = '../data/external/cb_2018_us_county_5m/cb_2018_us_county_5m.shp'
FIPS_PATH = '../data/external/counties_raw/cb_2023_us_county_5m.shp'
EAGLEI_DATA_PATH = '../data/raw/dynamic-rhythms-train-data/data/eaglei_data/'
NOAA_DATA_PATH = '../data/raw/dynamic-rhythms-train-data/data/NOAA_StormEvents/'
STORM_EVENTS_PATH = f'{NOAA_DATA_PATH}StormEvents_2014_2024.csv'

WORD_SIMILARITY_THRESHOLD = 0.8

## Configuration

In [7]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

warnings.filterwarnings('ignore')

## Functions

In [8]:
def word_similarity(w1, w2):
    similarity = SequenceMatcher(None, w1, w2).ratio()
    return similarity

## Read data

In [9]:
county = gpd.read_file(FIPS_PATH)
outages = ut.get_required_outages_dfs(EAGLEI_DATA_PATH, 2014, 2015, 2016)
storm_events = pd.read_csv(STORM_EVENTS_PATH)

Done reading.


IndexError: list index out of range

## Visualize Problem

In [31]:
# The GEOID feature is the fips code for county
county['fips'] = county.GEOID
fips_code__county_unique = county['fips'].unique()

In [33]:
fips_code__county_unique

array(['39071', '06003', '12033', ..., '48247', '29099', '13307'],
      shape=(3233,), dtype=object)

In [34]:
outages['fips'] = outages.fips_code.astype(str).str.zfill(5)
fips_code__outages_unique = outages['fips'].unique()

In [35]:
fips_code__outages_unique

array(['01037', '01051', '01109', ..., '02290', '38021', '47059'],
      shape=(2553,), dtype=object)

In [36]:
storm_events['fips'] = storm_events.STATE_FIPS.astype(str).str.zfill(2) + storm_events.CZ_FIPS.astype(str).str.zfill(3) 
fips_code__storm_events_unique = storm_events['fips'].unique()

In [37]:
fips_code__storm_events_unique

array(['33012', '25017', '29067', ..., '02823', '02813', '02807'],
      shape=(6105,), dtype=object)

In [38]:
### Outages minus official
len(set(fips_code__outages_unique) - set(fips_code__county_unique)) # Outage contains all the official fips

0

In [39]:
### Storm events minus official
len(set(fips_code__storm_events_unique) - set(fips_code__county_unique)) # There are about 3K of un-official fips. 

# Assumption: These 3K unofficial fips are typos or mistakes. 

2989

## Solve problem

In [45]:
county_columns = ['STATEFP', 'fips', 'NAME']
county_sol = county[county_columns].sample(3)

In [51]:
county_columns = ['STATEFP', 'fips', 'NAME']
county_sol = county[county_columns].drop_duplicates()

In [52]:
se_columns = ['STATE_FIPS', 'fips', 'CZ_NAME']
storm_sol = storm_events[se_columns].drop_duplicates()

In [55]:
storm_sol['STATEFP'] = storm_sol['STATE_FIPS'].astype(str).str.zfill(2)
storm_sol.drop('STATE_FIPS', axis=1, inplace=True)

In [106]:
solved_storm_sol = []
for state in county_sol.STATEFP.unique():
    county_sol_state = county_sol[county_sol.STATEFP==state]
    storm_sol_state = storm_sol[storm_sol.STATEFP==state]
    official_fips = county_sol_state.set_index('fips').NAME.to_dict()
    if storm_sol_state.shape[0] > 0:
        for fips_id, name in official_fips.items():
            name_condition1 = storm_sol_state.CZ_NAME.str.lower().str.contains(name.lower())
            name_condition2 = storm_sol_state.CZ_NAME.str.lower().apply(lambda x: word_similarity(x, name)) > WORD_SIMILARITY_THRESHOLD
            storm_sol_state.loc[((name_condition1)|(name_condition2)), 'new_fips'] = fips_id
        solved_storm_sol.append(storm_sol_state)

In [111]:
solved_storm_sol_df = pd.concat(solved_storm_sol)
solved_storm_sol_df['fips__cz_name'] = solved_storm_sol_df.fips + solved_storm_sol_df.CZ_NAME

In [114]:
storm_events['fips__cz_name'] = storm_events.fips + storm_events.CZ_NAME

In [124]:
storm_events_new_fips = storm_events.merge(solved_storm_sol_df[['fips__cz_name', 'new_fips']], on='fips__cz_name', how='left')

In [126]:
storm_events_new_fips.loc[storm_events_new_fips.new_fips.isna(), 'new_fips'] = storm_events_new_fips.loc[storm_events_new_fips.new_fips.isna()].fips  

In [129]:
fips_code__storm_events_new_unique = storm_events_new_fips['new_fips'].unique()

In [166]:
fips_code__storm_events_new_unique

array(['33011', '25017', '29067', ..., '02823', '02813', '02807'],
      shape=(4060,), dtype=object)

In [130]:
len(set(fips_code__storm_events_new_unique) - set(fips_code__county_unique)) # We have now corrected are about 2K of un-official fips. 

967

## Save solution

In [169]:
storm_events_new_fips.to_csv('../data/interim/storm_events_cleaned.csv')

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Continuation...?

In [141]:
remaining_steps = storm_events_new_fips[storm_events_new_fips.new_fips.isin(list(set(fips_code__storm_events_new_unique) - set(fips_code__county_unique)))]

In [145]:
remaining_steps['CZ_NAME2'] = remaining_steps['CZ_NAME'].str.strip() + ' '

In [150]:
grouped_remainding_steps = remaining_steps.groupby(['STATE_FIPS', 'CZ_FIPS']).CZ_NAME2.sum().reset_index()

In [161]:
grouped_remainding_steps['CZ_NAME_NEW'] = grouped_remainding_steps.CZ_NAME2.str.split(' ').apply(lambda x: [pd.Series(x).mode()])

In [162]:
grouped_remainding_steps

,STATE_FIPS,CZ_FIPS,CZ_NAME2,CZ_NAME_NEW
0,2,17,CAPE FAIRWEATHER TO CAPE SUCKLING COASTAL AREA...,[[CAPE]]
1,2,18,TAIYA INLET AND KLONDIKE HIGHWAY TAIYA INLET A...,"[[INLET, TAIYA]]"
2,2,21,EASTERN CHICHAGOF ISLAND EASTERN CHICHAGOF ISL...,"[[CHICHAGOF, EASTERN, ISLAND]]"
3,2,22,SALISBURY SOUND TO CAPE FAIRWEATHER COASTAL AR...,"[[AREA, CAPE, COASTAL, FAIRWEATHER, SALISBURY,..."
4,2,23,CAPE DECISION TO SALISBURY SOUND COASTAL AREA ...,"[[AREA, CAPE, COASTAL, DECISION, SALISBURY, SO..."
...,...,...,...,...
962,99,145,VEGA BAJA VEGA BAJA VEGA BAJA VEGA BAJA VEGA B...,"[[BAJA, VEGA]]"
963,99,147,VIEQUES VIEQUES VIEQUES,[[VIEQUES]]
964,99,149,VILLALBA VILLALBA VILLALBA VILLALBA VILLALBA V...,[[VILLALBA]]
965,99,151,YABUCOA YABUCOA YABUCOA YABUCOA YABUCOA YABUCO...,[[YABUCOA]]


In [165]:
county[county.STATEFP=='02'].sort_values('NAME')


,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,fips
2115,02,013,01419964,0500000US02013,02013,Aleutians East,04,18091255385,20800268276,"MULTIPOLYGON (((-159.32073 54.89727, -159.3135...",02013
205,02,016,01419965,0500000US02016,02016,Aleutians West,05,11375510886,25186009907,"MULTIPOLYGON (((179.48246 51.98283, 179.48656 ...",02016
1078,02,020,01416061,0500000US02020,02020,Anchorage,12,4419562836,615500204,"MULTIPOLYGON (((-150.068 61.16613, -150.00994 ...",02020
402,02,050,01419966,0500000US02050,02050,Bethel,05,105228817205,12768645652,"MULTIPOLYGON (((-165.41979 60.55242, -165.4151...",02050
2122,02,060,01419967,0500000US02060,02060,Bristol Bay,04,1248324005,949768219,"POLYGON ((-157.25146 58.62079, -157.17883 58.6...",02060
3054,02,068,01419988,0500000US02068,02068,Denali,04,32729577065,66105677,"POLYGON ((-152.99765 62.7274, -152.68015 62.98...",02068
1383,02,070,01419968,0500000US02070,02070,Dillingham,05,47478023796,5950680491,"MULTIPOLYGON (((-160.31736 58.69431, -160.3083...",02070
341,02,090,01419969,0500000US02090,02090,Fairbanks North Star,04,18983364355,273047179,"POLYGON ((-148.64821 64.59733, -148.64822 64.6...",02090
11,02,100,01419970,0500000US02100,02100,Haines,04,6059513753,1051059522,"MULTIPOLYGON (((-135.34934 59.02003, -135.3303...",02100
698,02,105,02371430,0500000US02105,02105,Hoonah-Angoon,05,16981566503,7801359160,"MULTIPOLYGON (((-133.59854 57.86028, -133.1769...",02105


## Conclusión:
Se logró reducir approx 60% de los errores de FIPS, clave que se utiliza para realizar la unión entre dataframes (por ahora).